In [99]:

import numpy as np
import pandas as pd
import networkx 
import pickle
import numpy as np

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [100]:
I = 5
df = pd.read_csv(f'./data/final_dataset_{I}.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees,Mutual_Follow,Adar_Index
0,Source,Target,1,0.001205,0.002399,-1,0,0,0,0,0,0,0,0.0
1,4135,291716,1,0.000640,0.001220,-1,0,0,0,0,0,0,0,0.0
2,102646,11606,1,0.000449,0.001668,-1,0,0,1,0,0,0,0,0.0
3,102646,291908,1,0.000449,0.000641,-1,0,0,0,0,0,0,0,0.0
4,14602,11267,1,0.000689,0.002050,5,0,1,2,0,0,0,0,0.0


In [101]:
# standarlize
x = df['Page_Rank_Src']
df['Page_Rank_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Page_Rank_Dst']
df['Page_Rank_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Shortest_Path']
df['Shortest_Path'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Dst']
df['Followees_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Src']
df['Followees_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Dst']
df['Followers_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Src']
df['Followers_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followees']
df['Int_Followees'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followers']
df['Int_Followers'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Mutual_Follow']
df['Int_Followers'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Adar_Index']
df['Int_Followers'] = ( x-x.min()) / ( x.max() - x.min() )
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees,Mutual_Follow,Adar_Index
0,Source,Target,1,0.136159,0.350920,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0
1,4135,291716,1,0.034455,0.138666,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0
2,102646,11606,1,0.000000,0.219206,0.000000,0.0,0.000000,0.333333,0.0,0.0,0.0,0,0.0
3,102646,291908,1,0.000000,0.034352,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0
4,14602,11267,1,0.043210,0.288001,0.461538,0.0,0.007692,0.666667,0.0,0.0,0.0,0,0.0


In [102]:

df_x = df.drop(columns=['Source', 'Destination', 'Class'])
df_y = df[['Class']]
x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = 0.3, random_state=42)

## Logistic Regression

In [103]:
penalty_values = ['l1','l2']
c_values = np.logspace(0,4,10)
param_grid = {
    'C':c_values,
    'penalty':penalty_values
}
gs_lr = GridSearchCV(cv=10, estimator=LogisticRegression(), param_grid = param_grid)

In [104]:
gs_lr.fit(x_train,y_train)

/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A colu

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                         'penalty': ['l1', 'l2']})

In [105]:
pickle.dump(gs_lr.best_estimator_, open(f'./data/Subgraph{I}/lr_model.p', 'wb')) 
print(gs_lr.best_params_)

{'C': 166.81005372000593, 'penalty': 'l2'}


In [106]:

print(classification_report(gs_lr.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       317
           1       0.96      1.00      0.98       306

    accuracy                           0.98       623
   macro avg       0.98      0.98      0.98       623
weighted avg       0.98      0.98      0.98       623



In [107]:
report = classification_report(gs_lr.best_estimator_.predict(x_test), y_test, output_dict=True)
df = pd.DataFrame(report).transpose()

df.to_csv(f'./data/Subgraph{I}/lr_classification_report.csv', index = False)

## Random Forest

In [108]:
estimators = [50, 100, 150]
depths = [9, 12, 15, 24]
split = np.random.randint(100, 150, 2)
leaves = np.random.randint(20, 30, 2)
param_grid = {'n_estimators' : estimators, 'max_depth' : depths, 'min_samples_split' : split, 'min_samples_leaf' : leaves}

gs_rf = GridSearchCV(cv = 10, estimator = RandomForestClassifier(), param_grid = param_grid)
gs_rf.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [9, 12, 15, 24],
                         'min_samples_leaf': array([27, 22]),
                         'min_samples_split': array([148, 110]),
                         'n_estimators': [50, 100, 150]})

In [109]:
pickle.dump(gs_rf.best_estimator_, open(f'./data/Subgraph{I}/rf_model.p', 'wb')) 
print(gs_rf.best_params_)

{'max_depth': 9, 'min_samples_leaf': 27, 'min_samples_split': 148, 'n_estimators': 50}


In [110]:

print(classification_report(gs_rf.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       316
           1       0.96      1.00      0.98       307

    accuracy                           0.98       623
   macro avg       0.98      0.98      0.98       623
weighted avg       0.98      0.98      0.98       623



In [111]:
report = classification_report(gs_rf.best_estimator_.predict(x_test), y_test, output_dict=True)
df = pd.DataFrame(report).transpose()

df.to_csv(f'./data/Subgraph{I}/rf_classification_report.csv', index = False)

## SVM

In [112]:
kernels = ['rbf','linear']
c_values = [0.001,0.01,1]
param_grid={'kernel':kernels, 'C':c_values}
gs_svm = GridSearchCV(cv=10, estimator=svm.SVC(), param_grid=param_grid)

In [113]:
gs_svm.fit(x_train,y_train)

/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A colu

GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 1], 'kernel': ['rbf', 'linear']})

In [114]:
pickle.dump(gs_svm.best_estimator_, open(f'./data/Subgraph{I}/svm_model.p', 'wb')) 
print(gs_svm.best_params_) 

{'C': 0.01, 'kernel': 'rbf'}


In [115]:

print(classification_report(gs_svm.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       1.00      0.95      0.97       320
           1       0.95      1.00      0.97       303

    accuracy                           0.97       623
   macro avg       0.97      0.97      0.97       623
weighted avg       0.98      0.97      0.97       623



In [116]:
report = classification_report(gs_svm.best_estimator_.predict(x_test), y_test, output_dict=True)
df = pd.DataFrame(report).transpose()

df.to_csv(f'./data/Subgraph{I}/svm_classification_report.csv', index = False)

## XGBoost

In [117]:
params = {
    'min_child_weight': [1,5,10],
    'gamma':[0.5,1,1.5,2,5],
    'subsample':[0.6,0.8,10],
    'cosample_bytree':[0.6,0.8,1.0],
    'max_depth':[3,4,5]
}
xgb = XGBClassifier(learning_rate = 0.02, n_estimators=600, objective = 'binary:logistic',silent=True, nthread =1)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1001)
gs_xgb = GridSearchCV(xgb, param_grid=params, n_jobs=4, cv=skf.split(x_train,y_train),verbose=3)
gs_xgb.fit(x_train,y_train)


Fitting 3 folds for each of 405 candidates, totalling 1215 fits
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.969 total time=   0.1s
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.969 total time=   0.1s
[CV 2/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.988 total time=   0.1s
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=10;, score=nan total time=   0.0s
[CV 2/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=10;, score=nan total time=   0.0s
[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.981 total time=   0.1s
[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=10;, score=nan total time=   0.0s
[CV 2/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_we

/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters

[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=10;, score=nan total time=   0.0s
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=0.8;, score=0.969 total time=   0.1s
[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=0.8;, score=0.981 total time=   0.1s
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=10;, score=nan total time=   0.0s
[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=0.6;, score=0.981 total time=   0.1s
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=10, subsample=0.6;, score=0.969 total time=   0.1s
[CV 2/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, subsample=0.8;, score=0.988 total time=   0.1s
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=10, subsample=10;, score=nan total time=   0.0s
[CV 2/3

/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters

[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.6;, score=0.969 total time=   0.1s
[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=1, subsample=0.8;, score=0.983 total time=   0.1s
[CV 1/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=1, subsample=10;, score=nan total time=   0.0s
[CV 2/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=1, subsample=10;, score=nan total time=   0.0s
[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=1, subsample=10;, score=nan total time=   0.0s
[CV 2/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.8;, score=0.988 total time=   0.1s
[CV 3/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=10;, score=nan total time=   0.0s
[CV 2/3] END cosample_bytree=0.6, gamma=0.5, max_depth=4, min_child_weight=5, subsample=0.6;, score=0.988 total time=   0.1s
[CV 3/3] END

...

/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters

[CV 2/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=0.8;, score=0.988 total time=   0.1s
[CV 3/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, subsample=0.6;, score=0.981 total time=   0.1s


/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "cosample_bytree", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/soojinlee/anaconda3/envs/myenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:52:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters

[CV 1/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=0.6;, score=0.969 total time=   0.1s
[CV 3/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, subsample=0.6;, score=0.981 total time=   0.1s
[CV 3/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=0.8;, score=0.981 total time=   0.1s
[CV 1/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=10;, score=nan total time=   0.0s
[CV 2/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=10;, score=nan total time=   0.0s
[CV 3/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=10;, score=nan total time=   0.0s
[CV 1/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, subsample=0.8;, score=0.969 total time=   0.0s
[CV 2/3] END cosample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, subsample=0.6;, score=0.988 total time=   0.0s
[CV 1/3] END cosample_by

GridSearchCV(cv=<generator object _BaseKFold.split at 0x1715989e0>,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     int...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=600,
                                     n_jobs=None, nthread=1,
                                     num_parallel_tree=None, ...),
             n_jobs=4,
             param_grid={'cosample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5, 2, 5], 'max_depth': [3, 4, 5],
                         'min_child_weight': [1, 5, 10],
                         'subsample': [0.6, 0.8, 10]},
             verbose=3)

In [118]:
pickle.dump(gs_xgb.best_estimator_, open(f'./data/Subgraph{I}/xgb_model.p', 'wb')) 
print(gs_xgb.best_params_) 

{'cosample_bytree': 0.6, 'gamma': 0.5, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.8}


In [119]:

print(classification_report(gs_xgb.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       310
           1       0.98      1.00      0.99       313

    accuracy                           0.99       623
   macro avg       0.99      0.99      0.99       623
weighted avg       0.99      0.99      0.99       623



In [120]:
report = classification_report(gs_xgb.best_estimator_.predict(x_test), y_test, output_dict=True)
df = pd.DataFrame(report).transpose()

df.to_csv(f'./data/Subgraph{I}/xgb_classification_report.csv', index = False)